In [1]:
import pandas as pd, numpy as np

In [2]:
import pickle

# LSFF: Output data counts under different binning scenarios

In [3]:
## read in data
path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_data_stage0_3_22_2021.csv'
df = pd.read_csv(path)

In [4]:
df.head()

,location_id,Unnamed: 10,compliance,compliance_source,date_recorded,fortification_standards,fortification_status,included,inclusion_justification,is_estimate,...,subnational_location_id,subnational_name,urbanicity,user,validated,value_025_percentile,value_975_percentile,value_description,value_mean,vehicle
0,163.0,NaN,NaN,NaN,10/30/2020,voluntary,survey answer,NaN,NaN,0.0,...,NaN,Rajasthan,mixed/both,nathaniel,Yes,21.1,27.9,percent of population eating fortified vehicle,24.3,oil
1,163.0,NaN,NaN,NaN,3/3/2021,voluntary,unknown,NaN,NaN,0.0,...,NaN,na,mixed/both,paulina,NaN,NaN,NaN,percent of vehicle that is fortified,10.0,oil
2,163.0,NaN,NaN,NaN,3/3/2021,voluntary,unknown,NaN,NaN,0.0,...,NaN,Bihar,rural,paulina,NaN,NaN,NaN,percent of vehicle that is fortified,0.0,oil
3,163.0,NaN,NaN,NaN,3/3/2021,voluntary,unknown,NaN,NaN,0.0,...,NaN,Odisha,rural,paulina,NaN,NaN,NaN,percent of vehicle that is fortified,0.0,oil
4,163.0,NaN,NaN,NaN,3/3/2021,voluntary,unknown,NaN,NaN,0.0,...,NaN,Rajasthan,rural,paulina,NaN,NaN,NaN,percent of vehicle that is fortified,14.0,oil


In [5]:
## for convenience
check_cols = ['location_id','location_name','vehicle','value_description','nutrient','value_mean','value_025_percentile',
       'value_975_percentile','notes']

In [8]:
df.loc[(df.location_name=="Vietnam") & (df.vehicle=="wheat flour"),['source_year','user'] + check_cols].sort_values("value_description")

,source_year,user,location_id,location_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,notes
588,2010,paulina,-1,Vietnam,wheat flour,percent of population eating vehicle,na,9.6,NaN,NaN,NaN
581,2010,paulina,-1,Vietnam,wheat flour,percent of population eating vehicle,na,4.2,NaN,NaN,NaN
583,2010,paulina,-1,Vietnam,wheat flour,percent of population eating vehicle,na,38.8,NaN,NaN,WRA defined as 19-50 years. Has other potentia...
584,2010,paulina,-1,Vietnam,wheat flour,percent of population eating vehicle,na,46.7,NaN,NaN,WRA defined as 19-50 years. Has other potentia...
585,2010,paulina,-1,Vietnam,wheat flour,percent of population eating vehicle,na,31.1,NaN,NaN,WRA defined as 19-50 years. Has other potentia...
586,2010,paulina,-1,Vietnam,wheat flour,percent of population eating vehicle,na,16.4,NaN,NaN,Under-5 defined as 6-60 months. Has other pote...
587,2010,paulina,-1,Vietnam,wheat flour,percent of population eating vehicle,na,17.5,NaN,NaN,NaN
589,2010,paulina,-1,Vietnam,wheat flour,percent of population eating vehicle,na,22.7,NaN,NaN,NaN
590,2010,paulina,-1,Vietnam,wheat flour,percent of population eating vehicle,na,13.5,NaN,NaN,Under-5 defined as 6-60 months. Has other pote...
591,2010,paulina,-1,Vietnam,wheat flour,percent of population eating vehicle,na,18.8,NaN,NaN,Under-5 defined as 6-60 months. Has other pote...


In [7]:
for i in df.loc[(df.location_name=="Vietnam") & (df.vehicle=="wheat flour"),['source_year','user'] + check_cols].sort_values("value_description").notes:
    print(i)

nan
nan
WRA defined as 19-50 years. Has other potentially usable data!
WRA defined as 19-50 years. Has other potentially usable data!
WRA defined as 19-50 years. Has other potentially usable data!
Under-5 defined as 6-60 months. Has other potentially usable data!
nan
nan
Under-5 defined as 6-60 months. Has other potentially usable data!
Under-5 defined as 6-60 months. Has other potentially usable data!
Source: Becky Tsang, Food Fortification Initiative. Personal communication. United States of America. 2017.
Source: Becky Tsang, Food Fortification Initiative. Personal communication. United States of America. 2017.
Source: Becky Tsang, Food Fortification Initiative. Personal communication. United States of America. 2017.
Source: Becky Tsang, Food Fortification Initiative. Personal communication. United States of America. 2017.
Source: Becky Tsang, Food Fortification Initiative. Personal communication. United States of America. 2017.
Assumed to be % of industrially processed wheat flour 

In [7]:
## check that we have nutrients iff expected

In [8]:
df[df.nutrient=='na'].value_description.unique()

array(['percent of population eating industrially produced vehicle',
       'percent of vehicle that is industrially produced',
       'percent of population eating vehicle',
       'percent of population eating adequately fortified vehicle'],
      dtype=object)

In [9]:
df[df.nutrient!='na'].value_description.unique()

array(['percent of population eating fortified vehicle',
       'percent of vehicle that is fortified',
       'percent of population eating adequately fortified vehicle'],
      dtype=object)

In [10]:
df[df.value_description=="percent of population eating adequately fortified vehicle"]

,location_id,Unnamed: 10,compliance,compliance_source,date_recorded,fortification_standards,fortification_status,included,inclusion_justification,is_estimate,...,subnational_location_id,subnational_name,urbanicity,user,validated,value_025_percentile,value_975_percentile,value_description,value_mean,vehicle
397,163.0,NaN,NaN,NaN,3/10/2021,voluntary,lab analysis,NaN,NaN,0.0,...,NaN,Rajasthan,mixed/both,paulina,NaN,NaN,NaN,percent of population eating adequately fortif...,0.1,wheat flour
399,163.0,NaN,NaN,NaN,3/10/2021,voluntary,lab analysis,NaN,NaN,0.0,...,NaN,Rajasthan,mixed/both,paulina,NaN,NaN,NaN,percent of population eating adequately fortif...,0.1,wheat flour
628,163.0,NaN,NaN,NaN,3/10/2021,voluntary,lab analysis,NaN,NaN,0.0,...,NaN,Rajasthan,mixed/both,paulina,NaN,NaN,NaN,percent of population eating adequately fortif...,0.1,wheat flour
630,163.0,NaN,NaN,NaN,3/10/2021,voluntary,lab analysis,NaN,NaN,0.0,...,NaN,Rajasthan,mixed/both,paulina,NaN,NaN,NaN,percent of population eating adequately fortif...,0.1,wheat flour
688,163.0,NaN,NaN,NaN,3/10/2021,voluntary,lab analysis,NaN,NaN,0.0,...,NaN,Rajasthan,mixed/both,paulina,NaN,NaN,NaN,percent of population eating adequately fortif...,0.1,wheat flour
690,163.0,NaN,NaN,NaN,3/10/2021,voluntary,lab analysis,NaN,NaN,0.0,...,NaN,Rajasthan,mixed/both,paulina,NaN,NaN,NaN,percent of population eating adequately fortif...,0.1,wheat flour
846,163.0,NaN,na,NaN,3/10/2021,na,lab analysis,NaN,NaN,0.0,...,NaN,Rajasthan,mixed/both,paulina,NaN,NaN,NaN,percent of population eating adequately fortif...,66.3,salt
849,163.0,NaN,na,NaN,3/10/2021,na,lab analysis,NaN,NaN,0.0,...,NaN,Rajasthan,mixed/both,paulina,NaN,NaN,NaN,percent of population eating adequately fortif...,66.3,salt


In [11]:
# relabel vals

#https://journals.sagepub.com/doi/pdf/10.1177/15648265120334S307
# cote divoire,
# "During the last year of the project, 206,410 MT of
# vegetable oil was marketed in Côte d’Ivoire. The fortified
# vegetable oil was covering 89% of the market."

val_descrip_map = {'percent of population eating fortified vehicle': 'percent of population eating fortified vehicle',
                   'percent of population eating adequately fortified vehicle': 'percent of population eating fortified vehicle',
 'percent of vehicle that is industrially produced': 'percent of vehicle that is industrially produced',
 'percent of vehicle that is fortified': 'percent of vehicle that is fortified',
 'percent of population eating industrially produced vehicle': 'percent of population eating industrially produced vehicle',
 'percent of marketshare of fortified products': 'percent of market covered by fortified product',
 'percent of population eating vehicle': 'percent of population eating vehicle'}

df.value_description = df.value_description.map(val_descrip_map)

In [12]:
df.value_description.unique()

array(['percent of population eating fortified vehicle',
       'percent of vehicle that is fortified',
       'percent of population eating industrially produced vehicle',
       'percent of vehicle that is industrially produced',
       'percent of population eating vehicle'], dtype=object)

In [13]:
def output_frame(cols= ['location_name','vehicle','nutrient','value_description']):
    """
    INPUT: desired columns to bin lsff data on
    ---
    OUTPUT: all legal value combinations using input columns
    ---
    eg: (iodine/salt) or (china/maize flour) are illegal combinations
    """
    
    # load legal pairs
    data_prep_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/'
    with open(data_prep_dir + 'lsff_vehicle_nutrient_pairs.pickle', 'rb') as handle:
        vehicle_nutrient_map = pickle.load(handle)

    with open(data_prep_dir + 'lsff_country_vehicle_pairs.pickle', 'rb') as handle:
        country_vehicle_map = pickle.load(handle)
        
    countries = [i for i in country_vehicle_map.keys() if type(i)==str]
    
    # value_descrips
    nutrient_relevant_vds = ['percent of population eating fortified vehicle',
           'percent of vehicle that is fortified',
           'percent of market covered by fortified product']
    nutrient_irrelevant_vds = ['percent of vehicle that is industrially produced',
       'percent of population eating industrially produced vehicle',
       'percent of population eating vehicle']

    estimate_status = [0,1]
    
    # all legal loc-vehicle-nutrient-val_ds for val_ds that have nutrient
    data_counts_a = pd.DataFrame([(i,j,k,l) for i in countries
                                  for j in country_vehicle_map[i]
                                  for k in vehicle_nutrient_map[j]
                                  for l in nutrient_relevant_vds],
                                 columns = ['location_name','vehicle','nutrient','value_description'])
    # all legal loc-vehicle-nutrient-val_ds for val_ds with nutrient='na'
    data_counts_b = pd.DataFrame([(i,j,'na',k) for i in countries
                                  for j in country_vehicle_map[i]
                                  for k in nutrient_irrelevant_vds],
                                 columns = ['location_name','vehicle','nutrient','value_description']) 
    
    data_counts = data_counts_a.append(data_counts_b).sort_values(by=['location_name','vehicle','value_description','nutrient'])
    
    return data_counts[cols].drop_duplicates().reset_index(drop=True)

In [14]:
def add_summary_measures_helper(frame, data, n_points = True, mean_mean = True, mean_values = True):
    """
    INPUT:
    - df with extracted data containing a 'value_mean' column, and a set of id_cols
    - a frame whose cols == id_cols
    - desired summary measures for each unique combination of id_cols
    """
    
    # merge extracted data onto frame containing desired bins
    merge_cols = frame.columns.tolist()
    data_counts = frame.merge(data[merge_cols + ['value_mean']], on = merge_cols, how = 'left')
    output = frame.copy()
    
    # calculate summary measures
    if n_points:
        data_counts_n = data_counts.groupby(merge_cols).count().rename(columns={'value_mean':'n_data_points'})
        output = output.merge(data_counts_n, on = merge_cols, how = 'left')
    if mean_mean:
        data_counts_mean = data_counts.groupby(merge_cols).mean().rename(columns={'value_mean':'mean_mean'})
        output = output.merge(data_counts_mean, on = merge_cols, how = 'left')
    if mean_values:
        data_counts_detail = data_counts.groupby(merge_cols).aggregate(lambda x : x.tolist()).rename(columns={'value_mean':'value_means'})
        output = output.merge(data_counts_detail, on = merge_cols, how = 'left')
        
    return output

In [15]:
def add_summary_measures_handling_estimates(frame, data, n_points = True, mean_mean = True, mean_values = True):
    """
    INPUT:
    - df with extracted data containing a 'value_mean' column, and a set of id_cols
    - a frame whose cols == id_cols
    - desired summary measures for each unique combination of id_cols
    -----
    @returns (df_A, df_B):
    df_A:
    - a row for each unique combination of id_cols
    - for each n_points, mean_mean, mean_values set to True, a col of corresponding vals
    - a column is_assumption, flagging rows for which we didn't have data, but made an assumption
    df_B:
    - all rows of dropped data.
    
    @effects
    - for any value where we have both data and an assumption, drops the assumption (and
    return as part of df_B)
    """
    
    #helpers
    merge_cols = frame.columns.tolist()
    return_cols = merge_cols
    
    #split out into extracted data vs assumptions
    extracted = add_summary_measures_helper(frame, df[df.is_estimate!=1], True, mean_mean, mean_values)
    assm = add_summary_measures_helper(frame, df[df.is_estimate==1], True, mean_mean, mean_values)
    
    #wide on is_estimated status
    data = extracted.merge(assm, on = merge_cols, suffixes = ['','_assm'], how = 'outer')
    assert(len(data)==len(extracted)), "merge issue"
    assert(len(data)==len(assm)), "merge issue"
    
    #rows for which we have both extracted and assumed "data"
    both = data[(data.n_data_points > 0) & (data.n_data_points_assm > 0)]
    if len(both)>0:
        print("(!) Make sure you are okay throwing out datapoint estimates for all rows for which we also have extracted data")
    
    #we will throw out the assumed data, and return this df to double check
    data.loc[(data.n_data_points > 0) & (data.n_data_points_assm > 0),'n_data_points_assum'] = 0
    
    
    #for all assumed data where we dont have extracted data, keep and mark as such
    data.loc[(data.n_data_points == 0) & (data.n_data_points_assm > 0),'is_assumption'] = 1
    data.loc[(data.is_assumption==1),'n_data_points'] = data.n_data_points_assm
    if n_points:
        return_cols = return_cols + ['n_data_points']
    if mean_mean:
        data.loc[(data.is_assumption==1),'mean_mean'] = data.mean_mean_assm
        return_cols = return_cols + ['mean_mean']
    if mean_values:
        data.loc[(data.is_assumption==1),'value_means'] = data.value_means_assm
        return_cols = return_cols + ['value_means']
    
    data.loc[data.is_assumption!=1,'is_assumption'] = 0
    data.is_estimate = data.is_assumption.astype(int)
        
    return data[return_cols + ['is_assumption']], both

## count data coverage for ALL county - vehicle - val_des - nutrient combinations

In [16]:
data_counts = output_frame(cols = ['location_name','vehicle','nutrient','value_description'])

In [17]:
data_counts_full, check = add_summary_measures_handling_estimates(frame = data_counts, data = df, n_points = True, mean_mean = True, mean_values = True)

(!) Make sure you are okay throwing out datapoint estimates for all rows for which we also have extracted data


/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:56: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [18]:
data_counts_full

,location_name,vehicle,nutrient,value_description,n_data_points,mean_mean,value_means,is_assumption
0,Angola,maize flour,folic acid,percent of market covered by fortified product,0,NaN,[nan],0.0
1,Angola,maize flour,iron,percent of market covered by fortified product,0,NaN,[nan],0.0
2,Angola,maize flour,vitamin a,percent of market covered by fortified product,0,NaN,[nan],0.0
3,Angola,maize flour,vitamin b1,percent of market covered by fortified product,0,NaN,[nan],0.0
4,Angola,maize flour,vitamin b12,percent of market covered by fortified product,0,NaN,[nan],0.0
...,...,...,...,...,...,...,...,...
1288,Vietnam,wheat flour,vitamin a,percent of vehicle that is fortified,1,0.0,[0.0],0.0
1289,Vietnam,wheat flour,vitamin b1,percent of vehicle that is fortified,1,0.0,[0.0],0.0
1290,Vietnam,wheat flour,vitamin b12,percent of vehicle that is fortified,1,0.0,[0.0],0.0
1291,Vietnam,wheat flour,zinc,percent of vehicle that is fortified,1,0.0,[0.0],0.0


In [19]:
#going to drop these vals where we assumed zero
check

,location_name,vehicle,nutrient,value_description,n_data_points,mean_mean,value_means,n_data_points_assm,mean_mean_assm,value_means_assm
458,Ethiopia,salt,folic acid,percent of population eating fortified vehicle,1,0.0,[0.0],1,0.0,[0.0]


In [20]:
data_counts_full_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/data_counts_full_3_22_2021.csv'
data_counts_full.to_csv(data_counts_full_path, index = False)

## count data coverage for target county - vehicle - val_des - nutrient combinations

In [21]:
target_value_descriptions = ['percent of population eating fortified vehicle',
       'percent of population eating industrially produced vehicle',
       'percent of population eating vehicle']

In [22]:
target_data_counts = data_counts_full[data_counts_full.value_description.isin(target_value_descriptions)]

In [23]:
target_data_counts

,location_name,vehicle,nutrient,value_description,n_data_points,mean_mean,value_means,is_assumption
6,Angola,maize flour,folic acid,percent of population eating fortified vehicle,0,NaN,[nan],0.0
7,Angola,maize flour,iron,percent of population eating fortified vehicle,0,NaN,[nan],0.0
8,Angola,maize flour,vitamin a,percent of population eating fortified vehicle,0,NaN,[nan],0.0
9,Angola,maize flour,vitamin b1,percent of population eating fortified vehicle,0,NaN,[nan],0.0
10,Angola,maize flour,vitamin b12,percent of population eating fortified vehicle,0,NaN,[nan],0.0
...,...,...,...,...,...,...,...,...
1281,Vietnam,wheat flour,vitamin b1,percent of population eating fortified vehicle,0,NaN,[nan],0.0
1282,Vietnam,wheat flour,vitamin b12,percent of population eating fortified vehicle,0,NaN,[nan],0.0
1283,Vietnam,wheat flour,zinc,percent of population eating fortified vehicle,0,NaN,[nan],0.0
1284,Vietnam,wheat flour,na,percent of population eating industrially prod...,0,NaN,[nan],0.0


In [24]:
target_data_counts_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/data_counts_ideal_bins_3_22_2021.csv'
target_data_counts.to_csv(target_data_counts_path, index = False)

## count data coverage for ALL county - vehicle - val_des combos, NOT nutrients

In [25]:
data_counts_nutrient_na = output_frame(cols = ['location_name','vehicle','value_description'])

In [26]:
data_counts_nutrient_na, check_b = add_summary_measures_handling_estimates(data_counts_nutrient_na, df, n_points = True, mean_mean = True, mean_values = True)

(!) Make sure you are okay throwing out datapoint estimates for all rows for which we also have extracted data


/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:56: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [27]:
data_counts_nutrient_na

,location_name,vehicle,value_description,n_data_points,mean_mean,value_means,is_assumption
0,Angola,maize flour,percent of market covered by fortified product,0,NaN,[nan],0.0
1,Angola,maize flour,percent of population eating fortified vehicle,0,NaN,[nan],0.0
2,Angola,maize flour,percent of population eating industrially prod...,0,NaN,[nan],0.0
3,Angola,maize flour,percent of population eating vehicle,0,NaN,[nan],0.0
4,Angola,maize flour,percent of vehicle that is fortified,6,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0
...,...,...,...,...,...,...,...
475,Vietnam,wheat flour,percent of population eating fortified vehicle,0,NaN,[nan],0.0
476,Vietnam,wheat flour,percent of population eating industrially prod...,0,NaN,[nan],0.0
477,Vietnam,wheat flour,percent of population eating vehicle,10,21.930000,"[4.2, 38.8, 46.7, 31.1, 16.4, 17.5, 9.6, 22.7,...",0.0
478,Vietnam,wheat flour,percent of vehicle that is fortified,7,5.357143,"[0.0, 37.5, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0


In [28]:
check_b

,location_name,vehicle,value_description,n_data_points,mean_mean,value_means,n_data_points_assm,mean_mean_assm,value_means_assm
169,Ethiopia,salt,percent of population eating fortified vehicle,2,42.8,"[85.6, 0.0]",1,0.0,[0.0]
223,India,salt,percent of population eating fortified vehicle,2,66.3,"[66.3, 66.3]",1,0.0,[0.0]
229,India,wheat flour,percent of population eating fortified vehicle,12,3.2,"[6.3, 0.1, 6.3, 0.1, 6.3, 0.1, 6.3, 0.1, 6.3, ...",1,0.0,[0.0]


In [29]:
data_counts_nutrient_na_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/data_counts_nutrient_na_3_22_2021.csv'
data_counts_nutrient_na.to_csv(data_counts_nutrient_na_path, index = False)